In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from skimage.feature import local_binary_pattern, hog, graycomatrix, graycoprops
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from xgboost import XGBClassifier
import pickle
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [17]:
def isskin(image):
    # test
    def nothing(x):
        pass

    # Load the image and convert to grayscale
    # gray = cv2.cvtColor(image, cv2.COLOR_BGRA2GRAY)
    # mean, std_dev = cv2.meanStdDev(gray)

    # Create a window with trackbars for parameter adjustment
    cv2.namedWindow('image')

    cv2.createTrackbar('HueMin', 'image', 0, 255, nothing)
    cv2.createTrackbar('HueMax', 'image', 20, 255, nothing)
    cv2.createTrackbar('SatMin', 'image', 48, 255, nothing)
    cv2.createTrackbar('SatMax', 'image', 255, 255, nothing)
    cv2.createTrackbar('ValMin', 'image', 80, 255, nothing)
    cv2.createTrackbar('ValMax', 'image', 255, 255, nothing)

    while True:
        # Get current positions of trackbars
        h_min = cv2.getTrackbarPos('HueMin', 'image')
        h_max = cv2.getTrackbarPos('HueMax', 'image')
        s_min = cv2.getTrackbarPos('SatMin', 'image')
        s_max = cv2.getTrackbarPos('SatMax', 'image')
        v_min = cv2.getTrackbarPos('ValMin', 'image')
        v_max = cv2.getTrackbarPos('ValMax', 'image')

        # Convert the image to HSV color space
        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        # Define lower and upper bounds for skin detection
        lower_hsv = np.array([h_min, s_min, v_min])
        upper_hsv = np.array([h_max, s_max, v_max])

        # Create a binary mask where skin regions are white
        skinMask = cv2.inRange(hsv_image, lower_hsv, upper_hsv)

        # Optional: Apply morphological operations to clean up the mask
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        skinMask = cv2.erode(skinMask, kernel, iterations=2)
        skinMask = cv2.dilate(skinMask, kernel, iterations=2)

        # Blur the mask to smooth the edges
        skinMask = cv2.GaussianBlur(skinMask, (5, 5), 0)

        # Apply the mask to the original image
        skin = cv2.bitwise_and(image, image, mask=skinMask)

        alpha = np.uint8(skinMask > 0) * 255
        result = cv2.merge((skin, alpha))

        cv2.imwrite("skin_detection_result.png", result)
        # result.istype()
        # Return the processed image
        return result

In [18]:
IMG_SIZE = 128

def extract_skin_and_preprocess(image_path):
    img = cv2.imread(str(image_path))
    if img is None:
        print(f"Failed to load {image_path}. Skipping.")
        return None
    skin_img = isskin(img)

    if skin_img.shape[2] == 4:
        skin_img = skin_img[:, :, :3] 
    denoised = cv2.fastNlMeansDenoising(skin_img, h=10)

    img_resized = cv2.resize(denoised, (IMG_SIZE, IMG_SIZE))

    # Convert to grayscale
    gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)

    return gray

In [19]:
def load_data(dataset_path):
    classes = {'Acne': 0, 'Bags': 1, 'Redness': 2}  # Map class names to labels
    X, y = [], []
    for class_name, label in classes.items():
        class_path = os.path.join(dataset_path, class_name)
        if not os.path.exists(class_path):
            print(f"Folder not found: {class_path}")
            continue
        for subfolder in os.listdir(class_path):  # Iterate through subfolders (0-10)
            subfolder_path = os.path.join(class_path, subfolder)
            if os.path.isdir(subfolder_path):
                for img_name in os.listdir(subfolder_path):  # Iterate through images
                    img_path = os.path.join(subfolder_path, img_name)
                    img = cv2.imread(img_path)
                    if img is not None:
                        img = extract_skin_and_preprocess(img_path)
                        X.append(img)
                        y.append(label)
                    else:
                        print(f"Failed to load image: {img_path}")
    print(f"Loaded {len(X)} images from {len(classes)} classes.")
    return np.array(X), np.array(y)

In [ ]:
dataset_path = 'Dataset'
X, y = load_data(dataset_path)

In [6]:
# LBP Feature Extraction
def extract_lbp_features(img, P=8, R=1):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray, P=P, R=R, method='uniform')
    hist, _ = np.histogram(lbp, bins=np.arange(0, P+3), range=(0, P+2))
    hist = hist.astype('float')
    hist /= (hist.sum() + 1e-6)  # Normalize
    return hist

# HOG Feature Extraction
def extract_hog_features(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features, _ = hog(gray, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, block_norm='L2-Hys', visualize=True)
    return features

# Color Histogram Feature Extraction
def extract_color_features(img, h_range=(0, 128), s_range=(100, 150), v_range=(0, 128)):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hist_h = cv2.calcHist([hsv], [0], None, [256], h_range)
    hist_s = cv2.calcHist([hsv], [1], None, [256], s_range)
    hist_v = cv2.calcHist([hsv], [2], None, [256], v_range)
    hist_h /= hist_h.sum() + 1e-6
    hist_s /= hist_s.sum() + 1e-6
    hist_v /= hist_v.sum() + 1e-6
    return np.concatenate([hist_h.flatten(), hist_s.flatten(), hist_v.flatten()])

# GLCM Feature Extraction
def extract_glcm_features(img, distances=[5], angles=[0], properties=('contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation')):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    glcm = graycomatrix(gray, distances=distances, angles=angles, levels=256, symmetric=True, normed=True)
    glcm_features = []
    for prop in properties:
        glcm_features.append(graycoprops(glcm, prop).flatten())
    return np.concatenate(glcm_features)

# Combined Feature Extraction
def extract_combined_features(img, h_range=(0, 128), s_range=(100, 150), v_range=(0, 128)):
    
    gray = extract_skin_and_preprocess(img)
    if gray is None:
        return None
    
    lbp_features = extract_lbp_features(gray)
    hog_features = extract_hog_features(gray)
    color_features = extract_color_features(cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR), h_range, s_range, v_range)
    # glcm_features = extract_glcm_features(img)
    return np.concatenate([lbp_features, hog_features, color_features])

# Example usage with training a model
def train_model_with_combined_features(X_images, y_labels, h_range=(0, 128), s_range=(100, 150), v_range=(0, 128)):
    # Extract features for all images
    X_features1 = np.array([extract_combined_features(img, h_range, s_range, v_range) for img in X_images])
    X_features2 = np.array([extract_glcm_features(img) for img in X_images])
    # Split into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_features1, y_labels, test_size=0.2, random_state=42)
    X2_train, X2_val, y2_train, y2_val = train_test_split(X_features2, y_labels, test_size=0.2, random_state=42)
    # Train a classifier (SVC)
    base_models = [
        ('svm', SVC(kernel='linear', probability=True)),
        ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
        ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42))
    ]
    # Meta-model
    meta_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
    
    # Stacking Classifier
    stacking_clf = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)
    stacking_clf2 = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)
    stacking_clf.fit(X_train, y_train)
    stacking_clf2.fit(X2_train, y2_train)
    
    # Evaluate model
    y2_pred = stacking_clf2.predict(X2_val)
    y_pred = stacking_clf.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    accuracy2 = accuracy_score(y2_val, y2_pred)
    print(f"Validation Accuracy Gabung semua: {accuracy}")
    print(f"Validation Accuracy GLCM Only: {accuracy2}")
    
    return stacking_clf


In [10]:
model = train_model_with_combined_features(X, y)

Failed to load [[[ 71  76  77]
  [ 68  72  72]
  [ 56  59  63]
  ...
  [ 17  20  25]
  [ 24  28  29]
  [ 26  30  31]]

 [[ 86  92  94]
  [ 85  90  93]
  [ 83  88  91]
  ...
  [ 30  34  35]
  [ 27  31  32]
  [ 28  32  34]]

 [[ 71  74  78]
  [ 72  77  80]
  [ 83  87  91]
  ...
  [ 29  33  34]
  [ 27  31  32]
  [ 27  31  32]]

 ...

 [[162 100  62]
  [160  98  60]
  [166 104  66]
  ...
  [151  85  44]
  [152  87  43]
  [152  87  43]]

 [[165 104  64]
  [162 101  61]
  [168 106  68]
  ...
  [150  82  39]
  [144  74  34]
  [158  89  49]]

 [[161 100  60]
  [161 100  60]
  [172 110  72]
  ...
  [155  87  44]
  [153  90  45]
  [154  89  46]]]. Skipping.
Failed to load [[[ 47  48  50]
  [ 42  44  45]
  [ 46  48  49]
  ...
  [ 49  50  54]
  [ 51  54  59]
  [ 48  51  54]]

 [[ 68  72  73]
  [ 66  70  71]
  [ 62  66  67]
  ...
  [ 59  64  67]
  [ 53  58  61]
  [ 57  60  65]]

 [[ 67  71  72]
  [ 54  58  59]
  [ 49  54  53]
  ...
  [ 25  28  32]
  [ 24  27  31]
  [ 28  31  35]]

 ...

 [[164 169 

KeyboardInterrupt: 

In [ ]:
file = open('model.pkl', 'wb')
pickle.dump(model, file)
file.close()